In [1]:
import pandas as pd
from scipy.stats import zscore
from sklearn.metrics.pairwise import euclidean_distances
from src.models import db, Laptop, Cpu, Gpu, Windows as Wind, Application as App
from src import app


### GET ALL LAPTOP FROM DATABASE

In [2]:
laptops_full = pd.read_csv("dev-data/data.test.csv")
laptops_full["totalStorage"] = laptops_full["SSD"] + laptops_full["HDD"]

laptops_full.head()

,no,name,CPU,GPU,maxSpeed,cores,gpuMaxSpeed,gpuMemory,directX,openGl,ram,SSD,HDD,Windows,buildNumber,Edition,totalStorage
0,1,Alienware m15 R3,Intel Core i7-10750H,NVIDIA GeForce RTX 2080 SUPER(Laptop),5.0,6.0,1740.0,8.0,12.1,4.6,32.0,1000.0,0,10,10240,HOME,1000.0
1,2,AORUS 15P,Intel Core i7-11800H,"NVIDIA GeForce RTX 3070 (Laptop, 130W)",4.6,8.0,1620.0,8.0,12.2,4.6,16.0,1000.0,0,10,10240,PRO,1000.0
2,3,AORUS 15P,Intel Core i7-11800H,"NVIDIA GeForce RTX 3070 (Laptop, 130W)",4.6,8.0,1620.0,8.0,12.2,4.6,16.0,2000.0,0,10,10240,HOME,2000.0
3,4,MSI GP66 Leopard,Intel Core i7-11800H,"NVIDIA GeForce RTX 3070 (Laptop, 130W)",4.6,8.0,1620.0,8.0,12.2,4.6,32.0,1000.0,0,10,10240,HOME,1000.0
4,5,MSI Katana 15,Intel Core i7-12650H,"NVIDIA GeForce RTX 4070 (Laptop, 105W)",4.7,10.0,2175.0,8.0,12.2,4.6,16.0,1000.0,0,10,10240,PRO,1000.0


### GET ALL APPS

In [3]:
app_query = db.select(App.id, App.name, App.minCpuSpeed, App.minCores, App.minDirectX, App.minOpenGl,
                      App.minGpuBoostClock, App.minGpuMemory, App.minRam, App.minStorage, Wind.buildNumber).join(App.windows)
with app.app_context():
    app_full = pd.read_sql(app_query, con=db.engine)

app_full.tail()

2024-04-16 23:34:22,888 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-04-16 23:34:22,890 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-16 23:34:22,895 INFO sqlalchemy.engine.Engine select current_schema()
2024-04-16 23:34:22,898 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-16 23:34:22,901 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-04-16 23:34:22,903 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-16 23:34:22,918 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-16 23:34:22,919 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

,id,name,minCpuSpeed,minCores,minDirectX,minOpenGl,minGpuBoostClock,minGpuMemory,minRam,minStorage,buildNumber
2386,52f51cb3-a5d1-4c8c-80e8-180c2e96cf08,Forspoken,3.40,6.0,12.05,4.50,589.0,2.048,8.0,50.0,7601
2387,9305e289-41ef-4bb4-a387-cdcdf794632d,Street Fighter 6,3.55,4.0,11.00,3.65,0.0,1.280,8.0,30.0,7601
2388,a34e29b1-2414-4d69-b491-aafa18b8fc2e,Starfield,3.40,6.0,12.05,4.50,589.0,2.048,8.0,50.0,7601
2389,4d421c6b-96c2-49f4-9cd6-15fea9cf669a,Star Wars Jedi: Survivor,3.30,4.0,11.10,4.25,0.0,1.024,8.0,55.0,7601
2390,91bf977d-ef0c-40ac-857e-52805b0b6dd1,Meet Your Maker,3.20,4.0,11.65,4.40,1114.0,2.048,8.0,20.0,10240


### CHOSING REQUIRED APPS

In [4]:
ids = ["89d1496b-988d-4e9e-a92b-da66ed7d893d", "395831aa-2f66-41b1-972d-f6356fe82344", "7a8921bd-5add-4b97-afb4-1bc21841e106", "ff6a43eb-75e9-47e5-b340-2a1b81afc595", "5ad5e08c-76c6-4d8a-8e03-5470d63537a8","4c73d3d3-1920-4ca9-87e2-341b64c2183d","32ac4365-7181-4246-9de4-92d497e742b5"]

app_full[app_full["name"].str.lower().str.contains("devil may")].head()


,id,name,minCpuSpeed,minCores,minDirectX,minOpenGl,minGpuBoostClock,minGpuMemory,minRam,minStorage,buildNumber
68,ff6a43eb-75e9-47e5-b340-2a1b81afc595,Devil May Cry 5,3.20,4.0,11.00,4.60,1032.0,2.000,8.0,35.0,10240
869,51894bd7-ce11-442e-922c-7a24439da756,Devil May Cry 4: Special Edition,2.65,2.0,10.05,2.70,0.0,0.384,2.0,27.0,7601
1106,4fbe4a42-76f2-4e15-994f-f660a9a264ec,DMC: Devil May Cry,2.60,2.0,10.05,2.70,0.0,0.288,2.0,9.0,2600
1264,697d6a71-7cb8-48d0-a5dd-872140bfe08f,Devil May Cry 4,2.40,1.0,9.00,1.75,0.0,0.256,0.0,8.0,2600
2278,6a48c3ed-3a65-42d4-9136-54706d7a3989,Devil May Cry HD Collection,2.65,2.5,11.10,4.30,982.5,2.560,4.0,12.0,7601


In [5]:
app_full.set_index("id", inplace=True)
choosen_apps = app_full.loc[ids]
choosen_apps.head()

,name,minCpuSpeed,minCores,minDirectX,minOpenGl,minGpuBoostClock,minGpuMemory,minRam,minStorage,buildNumber
id,,,,,,,,,,
89d1496b-988d-4e9e-a92b-da66ed7d893d,God of War,3.30,4.0,11.00,4.60,1178.0,4.000,8.0,70.0,10240
395831aa-2f66-41b1-972d-f6356fe82344,Touhou Danmaku Kagura Phantasia Lost,2.70,4.0,11.00,4.60,950.0,0.000,4.0,10.0,10240
7a8921bd-5add-4b97-afb4-1bc21841e106,Microsoft Office 2021,1.60,2.0,10.00,0.00,0.0,0.000,2.0,4.0,10240
ff6a43eb-75e9-47e5-b340-2a1b81afc595,Devil May Cry 5,3.20,4.0,11.00,4.60,1032.0,2.000,8.0,35.0,10240
5ad5e08c-76c6-4d8a-8e03-5470d63537a8,Metal Gear Rising: Revengeance,2.85,6.0,10.55,3.65,0.0,0.768,2.0,25.0,2600


### find the highest values of each property

In [6]:
sys_req = pd.DataFrame({
    "maxSpeed": [choosen_apps["minCpuSpeed"].max()],
    "cores": [choosen_apps["minCores"].max()],
    "gpuMaxSpeed": [choosen_apps["minGpuBoostClock"].max()],
    "gpuMemory": [choosen_apps["minGpuMemory"].max()],
    "openGl": [choosen_apps["minOpenGl"].max()],
    "directX": [choosen_apps["minDirectX"].max()],
    "ram": [choosen_apps["minRam"].max()],
    "totalStorage": [choosen_apps["minStorage"].sum()],
    "buildNumber": [choosen_apps["buildNumber"].max()],
})
sys_req.head()

,maxSpeed,cores,gpuMaxSpeed,gpuMemory,openGl,directX,ram,totalStorage,buildNumber
0,3.3,6.0,1524.5,5.12,4.6,12.05,12.0,214.0,10240


### DROP UNUSED LAPTOP PROPERTY

In [7]:
imp_laptop = laptops_full.drop(columns=["no", "CPU", "GPU", "Windows", "Edition", "name", "SSD", "HDD"])
imp_laptop.head()

KeyError: "['Name'] not found in axis"

### MERGE AND NORMALIZE SYSTEM REQUIREMENTS DATA AND LAPTOP DATA

In [ ]:
mixed_data = pd.concat([sys_req, imp_laptop], ignore_index=True)
mixed_data.head()

In [ ]:
normalized = mixed_data.apply(zscore)
normalized.head()

### CALCULATE EUCLIDEAN DISTANCE

calculate system requirements and laptop distance

In [ ]:
distance = euclidean_distances(normalized, normalized.values[0].reshape(1, -1))

In [ ]:
laptops_full["distance"] = distance[1:]
asc_data = laptops_full.sort_values(by="distance", ascending=True).drop(columns=["SSD", "HDD"])

print(sys_req)
asc_data.head()

### FILTER LAPTOP IF LOWER THAN SYSTEM REQUIREMENTS

In [ ]:
baseSpeed = (laptops_full["maxSpeed"] >= sys_req["maxSpeed"].values[0])
cores = (laptops_full["cores"] >= sys_req["cores"].values[0])
gpuMaxSpeed = (laptops_full["gpuMaxSpeed"] >= sys_req["gpuMaxSpeed"].values[0])
directX = (laptops_full["directX"] >= sys_req["directX"].values[0])
openGl = (laptops_full["openGl"] >= sys_req["openGl"].values[0])
totalStorage = (laptops_full["totalStorage"] >= sys_req["totalStorage"].values[0])
buildNumber = (laptops_full["buildNumber"] >= sys_req["buildNumber"].values[0])
vramFromRam = sys_req["gpuMemory"].values[0] - laptops_full["gpuMemory"]
laptops_full["vramFromRam"] = vramFromRam

# if vramFromRam is negative, then vram from ram is not needed (0) alocated vram from ram

laptops_full.loc[laptops_full["vramFromRam"] <= 0, "vramFromRam"] = 0
ram = (laptops_full["ram"] >= (sys_req["ram"].values[0] + laptops_full["vramFromRam"]))

filtered =  laptops_full.loc[baseSpeed & cores & gpuMaxSpeed & directX & openGl & ram & totalStorage & buildNumber]
filtered = filtered.drop(columns=["HDD", "SSD"])

print(sys_req)
print(choosen_apps["name"])
filtered.sort_values(by="distance", ascending=True).head()